In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

In [36]:
load = pd.read_csv('C:/Users/volco/AdvancedBA/Data/load.csv')
prices = pd.read_csv('C:/Users/volco/AdvancedBA/Data/prices.csv')
res_gen = pd.read_csv('C:/Users/volco/AdvancedBA/Data/res_gen.csv')
residual_gen = pd.read_csv('C:/Users/volco/AdvancedBA/Data/residual_generation.csv')

load['date'] = pd.to_datetime(load['date'])
prices['date'] = pd.to_datetime(prices['date'])
res_gen['date'] = pd.to_datetime(res_gen['date'])
residual_gen['date'] = pd.to_datetime(residual_gen['date'])

# Convert all price columns to numeric, coercing errors to NaN for non-numeric values in 'price_FRA'
for col in ['price_AT', 'price_BE', 'price_FRA', 'price_GER', 'price_NL']:
    prices[col] = pd.to_numeric(prices[col], errors='coerce')

missing_values_prices = prices.isnull().sum()

prices.fillna(prices.mean(), inplace=True)

In [42]:


# Load the datasets
load_data = pd.read_csv('C:/Users/volco/AdvancedBA/Data/load.csv', parse_dates=['date'], index_col='date')['load_GER']
prices_data = pd.read_csv('C:/Users/volco/AdvancedBA/Data/prices.csv', parse_dates=['date'], index_col='date')['price_GER']
res_gen_data = pd.read_csv('C:/Users/volco/AdvancedBA/Data/res_gen.csv', parse_dates=['date'], index_col='date')[['solar_forecastGER', 'windonshore_forecastGER']]

# Combine into a single DataFrame
data = pd.concat([load_data, prices_data, res_gen_data], axis=1).dropna()

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns, index=data.index)

# Assuming we've preprocessed and scaled our data, let's move to defining the environment



In [53]:
# class BatteryOptimizationEnv(gym.Env):
#     def __init__(self, data):
#         super(BatteryOptimizationEnv, self).__init__()
#         self.data = data
#         self.n_steps = data.shape[0]
#         self.current_step = 0
        
#         self.action_space = spaces.Discrete(3)  # Actions: 0 = Do nothing, 1 = Charge, 2 = Discharge
#         self.observation_space = spaces.Box(low=0, high=1, shape=(data.shape[1] + 1,), dtype=np.float32)  # State space + battery level
        
#         self.max_battery = 1.0  # Maximum normalized battery charge
#         self.battery_charge = self.max_battery / 2  # Start with half charge
#         self.charge_discharge_amount = 0.1  # Amount to charge or discharge at each action
        
#     def reset(self):
#         self.current_step = 0
#         self.battery_charge = self.max_battery / 2
#         return self._next_observation()  # Should return ONLY the observation.
    
    
#     def _next_observation(self):
#         obs = self.data.iloc[self.current_step].values
#         self.current_step += 1
#         return np.append(obs, self.battery_charge)
    
#     def step(self, action):
#         reward = 0
#         done = False
        
#         # Implement the logic for charging, discharging, or doing nothing
#         # Update the battery_charge and calculate the reward based on the action
#         # Simplified example:
#         if action == 1:
#             self.battery_charge = min(self.max_battery, self.battery_charge + self.charge_discharge_amount)
#             reward = -1  # Simplified reward, consider energy costs and savings
#         elif action == 2:
#             self.battery_charge = max(0, self.battery_charge - self.charge_discharge_amount)
#             reward = 1  # Simplified reward
        
#         if self.current_step >= self.n_steps:
#             done = True
        
#         return self._next_observation(), reward, done, {}


In [55]:


# # Ensure you have the correct version of gym installed
# # If not, you might need to install it via pip
# # e.g., !pip install gym

# # Load and preprocess your data here
# # Placeholder for data loading and preprocessing

# # Defining the Battery Optimization Environment
# class BatteryOptimizationEnv(gym.Env):
#     """A battery charge/discharge environment for OpenAI gym"""
#     metadata = {'render.modes': ['human']}

#     def __init__(self, data):
#         super(BatteryOptimizationEnv, self).__init__()
#         self.data = data
#         self.current_step = 0
#         self.done = False
#         self.max_battery = 100  # Maximum battery capacity in kWh
#         self.battery_level = self.max_battery / 2  # Starting battery level
#         self.charge_rate = 10  # kWh

#         # Define action and observation space
#         # Actions: 0 = Do Nothing, 1 = Charge, 2 = Discharge
#         self.action_space = spaces.Discrete(3)

#         # Observation space: normalized data features + battery level as a percentage of max capacity
#         self.observation_space = spaces.Box(low=0, high=1,
#                                             shape=(data.shape[1] + 1,), dtype=np.float32)

#     def _next_observation(self):
#         obs = np.append(self.data.iloc[self.current_step].values, self.battery_level / self.max_battery)
#         return obs

#     def step(self, action):
#         # Implement the logic to update the battery level based on the action
#         # Simplified reward logic based on action choice
#         reward = 0
#         if action == 1:  # Charging
#             self.battery_level = min(self.battery_level + self.charge_rate, self.max_battery)
#             reward = -1  # Example penalty for charging
#         elif action == 2:  # Discharging
#             self.battery_level = max(self.battery_level - self.charge_rate, 0)
#             reward = 1  # Example reward for discharging

#         self.current_step += 1

#         if self.current_step >= len(self.data):
#             self.done = True

#         return self._next_observation(), reward, self.done, {}

#     def reset(self):
#         self.current_step = 0
#         self.battery_level = self.max_battery / 2
#         self.done = False
#         return self._next_observation()

#     def render(self, mode='human', close=False):
#         # This method can be used for rendering the environment's state if necessary
#         print(f'Step: {self.current_step}, Battery Level: {self.battery_level} kWh')

# env = BatteryOptimizationEnv(data_scaled)


In [57]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class CustomEnv(gym.Env):
    def __init__(self):
        super(CustomEnv, self).__init__()
        self.action_space = spaces.Discrete(2)  # Example: 2 discrete actions
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)  # Example observation space

    def step(self, action):
        observation = np.array([0.0], dtype=np.float32)  # Placeholder observation, ensure dtype matches observation_space
        reward = 0  # Placeholder reward
        done = False  # Indicates if episode has ended
        truncated = False  # Indicates if episode was truncated (e.g., due to time limit)
        info = {}  # Additional info, can be empty
        return observation, reward, done, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)  # Optional: reset the environment's random state
        observation = np.array([0.0], dtype=np.float32)  # Reset observation, ensure dtype matches observation_space
        return observation, {}  # Reset returns observation and optionally an info dictionary

    def render(self, mode='human', close=False):
        pass  # Implement rendering if necessary for your environment

# Assuming stable_baselines3 and its env checker are compatible with gymnasium
from stable_baselines3.common.env_checker import check_env

env = CustomEnv()
check_env(env)  # Should now pass without assertion errors


In [58]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)


# To load the model
model = PPO.load("ppo_battery_optimization")
obs, _info = env.reset()  # Unpack the tuple to get the observation
for _ in range(1000):  # Adjust the number of steps as necessary
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, truncated, info = env.step(action)  # Ensure env.step() also returns the correct number of values
    env.render()  # Optionally render the environment's state if applicable
    if done:
        obs, _info = env.reset() 


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1486 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
--------------------------------------
| time/                   |          |
|    fps                  | 1017     |
|    iterations           | 2        |
|    time_elapsed         | 4        |
|    total_timesteps      | 4096     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.693   |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 10       |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
---------------------------------

In [45]:
# # Number of episodes for evaluation
# num_episodes = 100

# episode_rewards = []
# chosen_actions = []

# for episode in range(num_episodes):
#     obs, _info = env.reset()
#     done = False
#     episode_reward = 0
#     episode_actions = []
    
#     while not done:
#         action, _states = model.predict(obs, deterministic=True)
#         obs, reward, done, truncated, info = env.step(action)
        
#         episode_reward += reward
#         episode_actions.append(action)
    
#     episode_rewards.append(episode_reward)
#     chosen_actions.append(episode_actions)

# # Calculate and print the average reward per episode
# average_reward = sum(episode_rewards) / num_episodes
# print(f'Average Reward per Episode: {average_reward}')


In [59]:
# def evaluate_model(env, model, num_episodes=1):
#     episode_rewards = []
#     for episode in range(num_episodes):
#         obs, _ = env.reset()
#         done = False
#         total_rewards = 0
#         while not done:
#             action, _states = model.predict(obs, deterministic=True)
#             obs, rewards, done, truncated, info = env.step(action)
#             total_rewards += rewards
#             if done:
#                 break
#         episode_rewards.append(total_rewards)
#     average_reward = sum(episode_rewards) / num_episodes
#     print(f"Average reward over {num_episodes} episodes: {average_reward}")
#     return average_reward

# # Evaluate the model
# average_reward = evaluate_model(env, model, num_episodes=1)


In [48]:
# # Visualize the distribution of episode rewards
# plt.figure(figsize=(10, 6))
# plt.hist(episode_rewards, bins=20)
# plt.title('Distribution of Episode Rewards')
# plt.xlabel('Total Reward')
# plt.ylabel('Frequency')
# plt.show()

# # Analyze the distribution of actions
# # Assuming actions: 0 = Do Nothing, 1 = Charge, 2 = Discharge
# action_labels = ['Do Nothing', 'Charge', 'Discharge']
# action_counts = [sum(actions.count(action) for actions in chosen_actions) for action in range(len(action_labels))]

# plt.figure(figsize=(10, 6))
# plt.bar(action_labels, action_counts)
# plt.title('Distribution of Actions Taken')
# plt.xlabel('Action')
# plt.ylabel('Count')
# plt.show()

In [60]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class CustomEnv(gym.Env):
    """Custom environment for battery optimization"""
    def __init__(self):
        super(CustomEnv, self).__init__()
        self.action_space = spaces.Discrete(3)  # Actions: Do nothing, Charge, Discharge
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)
        # Initialize additional attributes here (e.g., battery level)

    def step(self, action):
        # Implement action logic, state transition, and reward calculation
        observation = np.array([0.0], dtype=np.float32)
        reward = 0
        done = False
        truncated = False
        info = {}
        return observation, reward, done, truncated, info

    def reset(self, seed=None, options=None):
        # Reset the environment state
        observation = np.array([0.0], dtype=np.float32)
        return observation, {}


In [62]:
# obs, _info = env.reset()  # Reset the environment to get the initial observation

# # Ensure obs is correctly shaped. This line might be redundant if obs is already in the correct shape.
# # It's provided here for demonstration purposes.
# obs = obs.reshape((5,))  # Adjust this reshape operation based on your actual observation space's expected shape

# for _ in range(1000):  # Adjust the number of steps as necessary
#     action, _states = model.predict(obs, deterministic=True)
    
#     obs, rewards, done, truncated, info = env.step(action)
#     # After env.step(), ensure obs is correctly shaped before the next predict() call
#     obs = obs.reshape((5,))  # Ensure this matches your observation space's shape
    
#     env.render()  # Optionally render the environment's state if applicable
    
#     if done:
#         obs, _info = env.reset()  # Reset the environment if done
#         obs = obs.reshape((5,))  # Ensure obs is correctly shaped after reset
